In [1]:

%matplotlib inline

import itertools
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow
ps = PorterStemmer()

In [2]:

data=pd.read_csv('train.csv', encoding = "ISO-8859-1")
#test_data=pd.read_csv('test.csv', encoding = "ISO-8859-1")
data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [3]:
def preProcessString(text):
    #Strips quotes at end of text
    text=text.strip('')
    #Rmoving twitter handles @user
    text=re.sub("@[\w]*"," ",text) 
    # Remove URLs with the space....
    text = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' ', text)
    # Strip space, " and ' from text
    text = text.strip(' "\'')
    #remove digits
    #text = re.sub(r'(\d)', '', text)
    #Remove repeated letters of string such as jusssssst to just 
    text=re.sub(r'(.)\1{3,}', r'\1', text)
    # remove all special characters
    text = re.sub('[^A-Za-z]', ' ', text)
    #replace two or more dots with space
    text = re.sub("\\.{2,}"," ",text);
    # converting all text into small letters and store them as words for further processing
    text_list = text.lower().split()
     # stemming the words (removing prefix and postfix) using Porter stemming algorithm..
    text_list = [ps.stem(word) for word in text_list]
    return ' '.join(text_list)

In [4]:
data['Preprocessed_data']=data['SentimentText'].apply(preProcessString)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
train, test = train_test_split(data, test_size=0.20)
Count_vectorization=TfidfVectorizer(use_idf=True, max_features=20000)

In [ ]:

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot


max_features = 2000
# cut texts after this number of words (among top max_features most common words)
maxlen = 200
batch_size = 32

print('Loading data...')
x_train = train["Preprocessed_data"]
y_train = train["Sentiment"]
x_test  = test["Preprocessed_data"]
y_test  = test["Sentiment"]

x_train = [one_hot(d, max_features) for d in x_train]
y_train = np.asarray(y_train)
x_test  = [one_hot(d, max_features) for d in x_test]
y_test  = np.asarray(y_test)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size, validation_split=0.2,
          epochs=15)
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)